In [3]:
import os, sys, json, csv
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import psychropy
%matplotlib inline
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

In [9]:
wrf_headers = ['PSFC', 'Q2', 'T2', 'RH', 'WINDS', 'GLW', 'SWDOWN', 'WINDD']
WRF_FOLDER = 'M02_EnergyPlus_Forcing_Historical'
heat_year = 2018
heat_month = [7]
heat_day_start = [2]
heat_day_end = [18]
# heat_month = [8, 9]
# heat_day_start = [20, 1]
# heat_day_end = [31, 29]
row = 384
col = 339

In [5]:
valid_wrf_grids = set()
with open(os.path.join('bldg-wrf-mapping','wrf-grids-2229-w-bldgs.geojson'), 'r') as f:
    data = json.load(f)
    
    for wrf_grid in data['features']:
        if wrf_grid['properties']['has_bldgs'] == 1:
            valid_wrf_grids.add(wrf_grid['properties']['id'])
        
len(valid_wrf_grids)

5806

## Write grid data (from txt) to timeseries CSV files

In [14]:
for wrf_header in wrf_headers:
    wrf_data = dict()
    year = str(heat_year)

    # M02_EnergyPlus_Forcing_Historical/SWDOWN/Variable_SWDOWN_2008070200.txt

    wrf_data = dict()
    for grid in valid_wrf_grids:
        wrf_data[grid] = []
    for mon in range(len(heat_month)):
        month = str(heat_month[mon]).zfill(2)
        for day in range(heat_day_start[mon], heat_day_end[mon] + 1):
            d = day - heat_day_start[mon]
            for h in range(24):
                date = str(day).zfill(2)
                hour = str(h).zfill(2)
                day = str(day).zfill(2)
                file_name = os.path.join(WRF_FOLDER, wrf_header, 'Variable_' + wrf_header + '_' + year + month + date + hour + '.txt')
                with open(file_name, 'r') as f:
                    for i in range(row):
                        cur_wrf_data = f.readline().strip().split('	') 
                        for j in range(col):
                            if i * col + j in valid_wrf_grids:
                                wrf_data[i * col + j].append(float(cur_wrf_data[j]))
    data = pd.DataFrame.from_dict(wrf_data)
    data.to_csv(os.path.join(WRF_FOLDER, 'time_series', wrf_header + '.csv'))

In [15]:
data.head() # WINDD

,32768,32769,32770,32771,32772,32773,32774,32775,32776,32777,...,32758,32759,32760,32761,32762,32763,32764,32765,32766,32767
0,75.331,75.331,75.331,75.331,75.331,75.331,75.331,75.331,75.331,75.331,...,75.331,75.331,75.331,75.331,75.331,75.331,75.331,75.331,75.331,75.331
1,79.602,79.602,79.602,79.602,79.602,79.602,79.602,79.602,79.602,79.602,...,79.602,79.602,79.602,79.602,79.602,79.602,79.602,79.602,79.602,79.602
2,81.279,81.279,81.279,81.279,81.279,81.279,81.279,81.279,81.279,81.279,...,81.279,81.279,81.279,81.279,81.279,81.279,81.279,81.279,81.279,81.279
3,75.475,75.475,75.475,75.475,75.475,75.475,75.475,75.475,75.475,75.475,...,75.475,75.475,75.475,75.475,75.475,75.475,75.475,75.475,75.475,75.475
4,84.902,84.902,84.902,84.902,84.902,84.902,84.902,84.902,84.902,84.902,...,84.902,84.902,84.902,84.902,84.902,84.902,84.902,84.902,84.902,84.902


In [16]:
valid_wrf_grids = list(valid_wrf_grids)

In [46]:
df_result_t2 = pd.read_csv(os.path.join(WRF_FOLDER, 'time_series', 'T2.csv'), sep=',', encoding='UTF-8', index_col=0)
df_result_p = pd.read_csv(os.path.join(WRF_FOLDER, 'time_series', 'PSFC.csv'), sep=',', encoding='UTF-8', index_col=0)
df_result_rh = pd.read_csv(os.path.join(WRF_FOLDER, 'time_series', 'RH.csv'), sep=',', encoding='UTF-8', index_col=0)
df_result_glw = pd.read_csv(os.path.join(WRF_FOLDER, 'time_series', 'GLW.csv'), sep=',', encoding='UTF-8', index_col=0)
df_result_wd = pd.read_csv(os.path.join(WRF_FOLDER, 'time_series', 'WINDD.csv'), sep=',', encoding='UTF-8', index_col=0)
df_result_ws = pd.read_csv(os.path.join(WRF_FOLDER, 'time_series', 'WINDS.csv'), sep=',', encoding='UTF-8', index_col=0)
df_result_q = pd.read_csv(os.path.join(WRF_FOLDER, 'time_series', 'Q2.csv'), sep=',', encoding='UTF-8', index_col=0)
df_result_rh.head(12)

,32768,32769,32770,32771,32772,32773,32774,32775,32776,32777,...,32758,32759,32760,32761,32762,32763,32764,32765,32766,32767
0,0.49324,0.49324,0.49324,0.49324,0.49324,0.49324,0.49324,0.49324,0.49324,0.49324,...,0.49324,0.49324,0.49324,0.49324,0.49324,0.49324,0.49324,0.49324,0.49324,0.49324
1,0.55147,0.55147,0.55147,0.55147,0.55147,0.55147,0.55147,0.55147,0.55147,0.55147,...,0.55147,0.55147,0.55147,0.55147,0.55147,0.55147,0.55147,0.55147,0.55147,0.55147
2,0.64365,0.64365,0.64365,0.64365,0.64365,0.64365,0.64365,0.64365,0.64365,0.64365,...,0.64365,0.64365,0.64365,0.64365,0.64365,0.64365,0.64365,0.64365,0.64365,0.64365
3,0.77214,0.77214,0.77214,0.77214,0.77214,0.77214,0.77214,0.77214,0.77214,0.77214,...,0.77214,0.77214,0.77214,0.77214,0.77214,0.77214,0.77214,0.77214,0.77214,0.77214
4,0.85823,0.85823,0.85823,0.85823,0.85823,0.85823,0.85823,0.85823,0.85823,0.85823,...,0.85823,0.85823,0.85823,0.85823,0.85823,0.85823,0.85823,0.85823,0.85823,0.85823
5,0.89079,0.89079,0.89079,0.89079,0.89079,0.89079,0.89079,0.89079,0.89079,0.89079,...,0.89079,0.89079,0.89079,0.89079,0.89079,0.89079,0.89079,0.89079,0.89079,0.89079
6,0.88416,0.88416,0.88416,0.88416,0.88416,0.88416,0.88416,0.88416,0.88416,0.88416,...,0.88416,0.88416,0.88416,0.88416,0.88416,0.88416,0.88416,0.88416,0.88416,0.88416
7,0.89602,0.89602,0.89602,0.89602,0.89602,0.89602,0.89602,0.89602,0.89602,0.89602,...,0.89602,0.89602,0.89602,0.89602,0.89602,0.89602,0.89602,0.89602,0.89602,0.89602
8,0.89119,0.89119,0.89119,0.89119,0.89119,0.89119,0.89119,0.89119,0.89119,0.89119,...,0.89119,0.89119,0.89119,0.89119,0.89119,0.89119,0.89119,0.89119,0.89119,0.89119
9,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,...,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549


### Convert UTC to local time. Do the same manually for solar.

In [47]:
timeshift = 8
total_num_days = 18 - 2
df_result_t2 = df_result_t2[timeshift + 1: timeshift + 1 + 24 * total_num_days]
df_result_p = df_result_p[timeshift + 1: timeshift + 1 + 24 * total_num_days]
df_result_rh = df_result_rh[timeshift + 1: timeshift + 1 + 24 * total_num_days]
df_result_glw = df_result_glw[timeshift + 1: timeshift + 1 + 24 * total_num_days]
df_result_wd = df_result_wd[timeshift + 1: timeshift + 1 + 24 * total_num_days]
df_result_ws = df_result_ws[timeshift + 1: timeshift + 1 + 24 * total_num_days]
df_result_q = df_result_q[timeshift + 1: timeshift + 1 + 24 * total_num_days]
df_result_t2 = df_result_t2.reset_index(drop=True)
df_result_p = df_result_p.reset_index(drop=True)
df_result_rh = df_result_rh.reset_index(drop=True)
df_result_glw = df_result_glw.reset_index(drop=True)
df_result_wd = df_result_wd.reset_index(drop=True)
df_result_q = df_result_q.reset_index(drop=True)
df_result_ws = df_result_wd.reset_index(drop=True)
df_result_rh.head(12)

,32768,32769,32770,32771,32772,32773,32774,32775,32776,32777,...,32758,32759,32760,32761,32762,32763,32764,32765,32766,32767
0,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,...,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549,0.89549
1,0.91244,0.91244,0.91244,0.91244,0.91244,0.91244,0.91244,0.91244,0.91244,0.91244,...,0.91244,0.91244,0.91244,0.91244,0.91244,0.91244,0.91244,0.91244,0.91244,0.91244
2,0.90958,0.90958,0.90958,0.90958,0.90958,0.90958,0.90958,0.90958,0.90958,0.90958,...,0.90958,0.90958,0.90958,0.90958,0.90958,0.90958,0.90958,0.90958,0.90958,0.90958
3,0.89747,0.89747,0.89747,0.89747,0.89747,0.89747,0.89747,0.89747,0.89747,0.89747,...,0.89747,0.89747,0.89747,0.89747,0.89747,0.89747,0.89747,0.89747,0.89747,0.89747
4,0.89633,0.89633,0.89633,0.89633,0.89633,0.89633,0.89633,0.89633,0.89633,0.89633,...,0.89633,0.89633,0.89633,0.89633,0.89633,0.89633,0.89633,0.89633,0.89633,0.89633
5,0.78048,0.78048,0.78048,0.78048,0.78048,0.78048,0.78048,0.78048,0.78048,0.78048,...,0.78048,0.78048,0.78048,0.78048,0.78048,0.78048,0.78048,0.78048,0.78048,0.78048
6,0.75929,0.75929,0.75929,0.75929,0.75929,0.75929,0.75929,0.75929,0.75929,0.75929,...,0.75929,0.75929,0.75929,0.75929,0.75929,0.75929,0.75929,0.75929,0.75929,0.75929
7,0.68146,0.68146,0.68146,0.68146,0.68146,0.68146,0.68146,0.68146,0.68146,0.68146,...,0.68146,0.68146,0.68146,0.68146,0.68146,0.68146,0.68146,0.68146,0.68146,0.68146
8,0.62157,0.62157,0.62157,0.62157,0.62157,0.62157,0.62157,0.62157,0.62157,0.62157,...,0.62157,0.62157,0.62157,0.62157,0.62157,0.62157,0.62157,0.62157,0.62157,0.62157
9,0.59036,0.59036,0.59036,0.59036,0.59036,0.59036,0.59036,0.59036,0.59036,0.59036,...,0.59036,0.59036,0.59036,0.59036,0.59036,0.59036,0.59036,0.59036,0.59036,0.59036


In [48]:
df_result_q.describe()

,32768,32769,32770,32771,32772,32773,32774,32775,32776,32777,...,32758,32759,32760,32761,32762,32763,32764,32765,32766,32767
count,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,...,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000,384.000000
mean,0.008753,0.008753,0.008753,0.008753,0.008753,0.008753,0.008753,0.008753,0.008753,0.008753,...,0.008753,0.008753,0.008753,0.008753,0.008753,0.008753,0.008753,0.008753,0.008753,0.008753
std,0.001975,0.001975,0.001975,0.001975,0.001975,0.001975,0.001975,0.001975,0.001975,0.001975,...,0.001975,0.001975,0.001975,0.001975,0.001975,0.001975,0.001975,0.001975,0.001975,0.001975
min,0.002843,0.002843,0.002843,0.002843,0.002843,0.002843,0.002843,0.002843,0.002843,0.002843,...,0.002843,0.002843,0.002843,0.002843,0.002843,0.002843,0.002843,0.002843,0.002843,0.002843
25%,0.007918,0.007918,0.007918,0.007918,0.007918,0.007918,0.007918,0.007918,0.007918,0.007918,...,0.007918,0.007918,0.007918,0.007918,0.007918,0.007918,0.007918,0.007918,0.007918,0.007918
50%,0.009096,0.009096,0.009096,0.009096,0.009096,0.009096,0.009096,0.009096,0.009096,0.009096,...,0.009096,0.009096,0.009096,0.009096,0.009096,0.009096,0.009096,0.009096,0.009096,0.009096
75%,0.010180,0.010180,0.010180,0.010180,0.010180,0.010180,0.010180,0.010180,0.010180,0.010180,...,0.010180,0.010180,0.010180,0.010180,0.010180,0.010180,0.010180,0.010180,0.010180,0.010180
max,0.012343,0.012343,0.012343,0.012343,0.012343,0.012343,0.012343,0.012343,0.012343,0.012343,...,0.012343,0.012343,0.012343,0.012343,0.012343,0.012343,0.012343,0.012343,0.012343,0.012343


### This solar data is manually parsed (dir-dif separation) using the external Excel tool. Using one data point only.

In [49]:
df_wrf_data = pd.read_csv(os.path.join(WRF_FOLDER, 'time_series', 'LA-SOLAR.csv'), sep=',', encoding='UTF-8')
df_wrf_data.head()

,mon,day,hour,sw_down,sw_normal,sw_dif,sw_dir
0,7,2,0,0.0,0.0,0.0,0.0
1,7,2,1,0.0,0.0,0.0,0.0
2,7,2,2,0.0,0.0,0.0,0.0
3,7,2,3,0.0,0.0,0.0,0.0
4,7,2,4,0.0,0.0,0.0,0.0


In [50]:
for i in range(len(valid_wrf_grids)):
    grid_id = str(valid_wrf_grids[i])
    df_wrf_data_new = df_wrf_data
    df_wrf_data_new['dry-bulb'] = df_result_t2[grid_id]
    df_wrf_data_new['rel-humidity'] = df_result_rh[grid_id] * 100
    df_wrf_data_new['hum-rat'] = df_result_q[grid_id]
    df_wrf_data_new['pressure'] = df_result_p[grid_id]
    df_wrf_data_new['dw-lwr'] = df_result_glw[grid_id]
    df_wrf_data_new['wind-dir'] = df_result_wd[grid_id]
    df_wrf_data_new['wind-speed'] = df_result_ws[grid_id]
    df_wrf_data_new.to_csv(os.path.join(WRF_FOLDER, 'grids_csv', grid_id + '.csv'))

In [51]:
START_MON = '7'
START_DAY = '2'
END_MON = '7'
END_DAY = '28'
LOC_WEATHER_FILE = os.path.join(WRF_FOLDER, 'USA_CA_Los.Angeles.Intl.AP.722950_TMY3.epw')
csv_headers_epw_map = {6: 'dry-bulb', 7:'dew-point', 8:'rel-humidity', 9:'pressure', 12: 'dw-lwr',
                        13: 'sw_normal', 14: 'sw_dir', 15: 'sw_dif', 20:'wind-dir', 21:'wind-speed'}
days_in_mon = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

## Write CSV to EPW

In [52]:
## write EPW example
grid_id = str(valid_wrf_grids[0])
df_csv_result = pd.read_csv(os.path.join(WRF_FOLDER, 'grids_csv', grid_id + '.csv'), sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
data_to_write = dict()
for i, r in df_csv_result.iterrows():
    r['dew-point'] = psychropy.Dew_point(float(r['pressure'])/1000.0, float(r['hum-rat']))
    mon = int(r['mon'])
    day = int(r['day'])
    hour = int(r['hour'])
    data_to_write[str(mon) + "/" + str(day) + "/" + str(hour)] = r

In [53]:
data_to_write['7/2/1']

Unnamed: 0          1.000000
mon                 7.000000
day                 2.000000
hour                1.000000
sw_down             0.000000
sw_normal           0.000000
sw_dif              0.000000
sw_dir              0.000000
dry-bulb            8.999400
rel-humidity       91.244000
hum-rat             0.006523
pressure        99806.000000
dw-lwr            303.590000
wind-dir          180.500000
wind-speed        180.500000
dew-point           7.513548
Name: 1, dtype: float64

In [59]:
for i in range(10):
# for i in range(len(valid_wrf_grids)):
    grid_id = str(valid_wrf_grids[i])
    df_csv_result = pd.read_csv(os.path.join(WRF_FOLDER, 'grids_csv', grid_id + '.csv'), sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
    data_to_write = dict()
    for i, r in df_csv_result.iterrows():
        try:
            r['dew-point'] = psychropy.Dew_point(float(r['pressure']) / 1000.0, float(r['hum-rat']))
        except:
            print('Something wrong with dew point calculation.')
            r['dew-point'] = None
        mon = int(r['mon'])
        day = int(r['day'])
        hour = int(r['hour'])
        data_to_write[str(mon) + "/" + str(day) + "/" + str(hour)] = r

    EPW_FILE = os.path.join(WRF_FOLDER, 'wrf_epw', grid_id + '.epw')
    if os.path.exists(EPW_FILE):
        continue
    f = open(EPW_FILE, 'w')
    with open(LOC_WEATHER_FILE, newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in csvreader:
            if (len(row) == 35):
                mon = row[1]
                day = row[2]
                hour = row[3]
                minute = row[4]
                # parse 8/1/18 0:00 to 7/31/18 24:00
                if int(hour) == 24:
                    hour = 0
                    if int(day) == days_in_mon[int(mon)]:
                        mon = int(mon) + 1
                        day = 1
                    else:
                        day = int(day) + 1
                cur_time_stamp = str(mon) + "/" + str(day) + "/" + str(hour)
                if cur_time_stamp in data_to_write:
                    cur_data_to_write = data_to_write[cur_time_stamp]
                    for epw_col in csv_headers_epw_map:
                        if csv_headers_epw_map[epw_col] in cur_data_to_write:
                            row[epw_col] = str(round(cur_data_to_write[csv_headers_epw_map[epw_col]], 3))
            f.write(','.join(row) + '\n')
    f.close()